In [ ]:
#!pip install konlpy
import pandas as pd
import numpy as np
import re
import urllib.request
import time
#from konlpy.tag import Okt

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f36580f0410>)

In [ ]:
data1 = pd.read_table('ratings_train.txt')
data2 = pd.read_table('ratings_test.txt')
#test_data = pd.read_table('ratings_test.txt')
print(len(data1)) # 훈련용 리뷰 개수 출력
data1 = pd.DataFrame(data1)
print(len(data2)) # 훈련용 리뷰 개수 출력
data2 = pd.DataFrame(data2)
data = pd.concat([data1, data2])
print(len(data))
data.head()
#print('테스트용 리뷰 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력
#print(test_data[:5])

150000
50000
200000


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
data = pd.read_csv('/content/drive/MyDrive/텍마팀플/채널모든댓글.csv',index_col=0)
data = data.drop(['token'],axis=1)
data = data.drop(['topic'],axis=1)
data['label'] = 1
data
'''

In [ ]:
data['document'].nunique()
data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
print(data.groupby('label').size().reset_index(name = 'count'))

   label  count
0      0  97375
1      1  97169


In [ ]:
!git clone https://github.com/ssut/py-hanspell.git # 네이버 맞춤법 검사기

Cloning into 'py-hanspell'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 101 (delta 5), reused 10 (delta 3), pack-reused 81
Receiving objects: 100% (101/101), 25.27 KiB | 12.64 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [ ]:
cd py-hanspell/

/content/py-hanspell


In [ ]:
!python setup.py install

In [ ]:
# 한글과 공백을 제외하고 모두 제거
data['document'] = data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
data['document'] = data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
data = data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(data.isnull().values.any()) # Null 값이 존재하는지 확인
print(data.isnull().sum())
print(len(data))
data.head()

False
id          0
document    0
label       0
dtype: int64
193518


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [ ]:
#학습,테스트 데이터 분리
sentiments = data['label']
reviews = data['document']

train_review, test_review, train_senti, test_senti = train_test_split(reviews,
                                                                      sentiments,
                                                                      test_size = 0.3,
                                                                      random_state = 42)

In [ ]:
print("train :",len(train_review))
print("test :",len(test_review))
stop_list = ['의','가','이','은','들','는','좀','걍','과','도','를','으로','자','에','와','하다']

train : 135462
test : 58056


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(stop_words=stop_list)),
    ('lr_clf', LogisticRegression(solver='liblinear'))
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_)2개를 연달아 붙여 GridSearchCV에 사용될 
# 파라미터/하이퍼 파라미터 이름과 값을 설정. . 
params = { 'cnt_vect__ngram_range': [(1,1), (1,2), (1,3)],
           #'tfidf_vect__max_df': [100, 300, 700],
           'lr_clf__C': [1,5,10]
}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV( pipeline,param_grid =params, cv=3 , scoring='accuracy',verbose=1)
start_time = time.time()
grid_cv_pipe.fit(train_review , train_senti)
print(grid_cv_pipe.best_score_ , grid_cv_pipe.best_params_)

pred = grid_cv_pipe.predict(test_review)
pred_proba = pipeline.predict_proba(test_review)[:,1]
print('Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(test_senti ,pred)))
print('Logistic Regression 의 AUC Score {0:.3f}'.format(roc_auc_score(test_senti ,pred_proba)))
print('Logistic Regression 수행 시간: {:.1f}초'.format(time.time() - start_time))

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  4.6min finished


0.796238059381967 {'cnt_vect__ngram_range': (1, 2), 'lr_clf__C': 10}


NotFittedError: ignored

In [ ]:

#countvector - Logistic Regression
pipeline = Pipeline([('cnt_vect',CountVectorizer(stop_words = stop_list,
                                                ngram_range = (1,2))),
                     ('lr_clf',LogisticRegression(C=10))])
start_time = time.time()
pipeline.fit(train_review,train_senti)
pred = pipeline.predict(test_review)
pred_proba = pipeline.predict_proba(test_review)[:,1]

acc = accuracy_score(test_senti,pred)
auc = roc_auc_score(test_senti,pred_proba)
print(f"정확도 :{acc: .4f}\nAUC score:{auc: .4f}")
print('Logistic Regression 수행 시간: {:.1f}초'.format(time.time() - start_time))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


정확도 : 0.7964
AUC score: 0.8908
Logistic Regression 수행 시간: 35.2초


In [ ]:
#tfidf - logistic Regression
pipeline = Pipeline([('tfidf',TfidfVectorizer(stop_words = stop_list,
                                                ngram_range = (1,2))),
                     ('lr_clf',LogisticRegression(C=10))])
start_time = time.time()
pipeline.fit(train_review,train_senti)
pred = pipeline.predict(test_review)
pred_proba = pipeline.predict_proba(test_review)[:,1]

acc = accuracy_score(test_senti,pred)
auc = roc_auc_score(test_senti,pred_proba)
print(f"정확도 :{acc: .4f}\nAUC score:{auc: .4f}")
print('Logistic Regression 수행 시간: {:.1f}초'.format(time.time() - start_time))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


정확도 : 0.8000
AUC score: 0.8919
Logistic Regression 수행 시간: 36.4초


In [ ]:
#tfidf - naive bayes
pipeline = Pipeline([('tfidf',TfidfVectorizer(stop_words = stop_list,
                                                ngram_range = (1,2))),
                     ('bnb',BernoulliNB())])
start_time = time.time()
pipeline.fit(train_review,train_senti)
pred = pipeline.predict(test_review)
pred_proba = pipeline.predict_proba(test_review)[:,1]

acc = accuracy_score(test_senti,pred)
auc = roc_auc_score(test_senti,pred_proba)
print(f"정확도 :{acc: .4f}\nAUC score:{auc: .4f}")
print('naive bayes 수행 시간: {:.1f}초'.format(time.time() - start_time))

정확도 : 0.8139
AUC score: 0.9030
naive bayes 수행 시간: 11.3초


In [ ]:
#tfidf - knn
pipeline = Pipeline([('tfidf',TfidfVectorizer(stop_words = stop_list,
                                                ngram_range = (1,2))),
                     ('knc',KNeighborsClassifier(n_neighbors=1))])
start_time = time.time()
pipeline.fit(train_review,train_senti)
pred = pipeline.predict(test_review)
pred_proba = pipeline.predict_proba(test_review)[:,1]

acc = accuracy_score(test_senti,pred)
auc = roc_auc_score(test_senti,pred_proba)
print(f"정확도 :{acc: .4f}\nAUC score:{auc: .4f}")
print('knn 수행 시간: {:.1f}초'.format(time.time() - start_time))

정확도 : 0.5852
AUC score: 0.5844
knn 수행 시간: 179.7초


In [ ]:
#tfidf - multilayer perceptron
pipeline = Pipeline([('tfidf',TfidfVectorizer(stop_words = stop_list,
                                                ngram_range = (1,2))),
                     ('mlpc',MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(5,2),random_state=0))])
start_time = time.time()
pipeline.fit(train_review,train_senti)
pred = pipeline.predict(test_review)
pred_proba = pipeline.predict_proba(test_review)[:,1]

acc = accuracy_score(test_senti,pred)
auc = roc_auc_score(test_senti,pred_proba)
print(f"정확도 :{acc: .4f}\nAUC score:{auc: .4f}")
print('mlp 수행 시간: {:.1f}초'.format(time.time() - start_time))

정확도 : 0.7576
AUC score: 0.8428
mlp 수행 시간: 45.8초
